## Detección de objetos en vídeos con OpenCV DNN 

In [6]:
#Librerias necesarias para el procesamiento de video
import cv2
import numpy as np
import time

**Predicción sobre fotogramas de video**

In [7]:
# Cargar los nombres de las clases COCO
with open('input/object_detection_classes_coco.txt', 'r') as f:
   class_names = f.read().split('\n')
 
# Obtener una matriz de colores diferente para cada una de las clases
COLORS = np.random.uniform(0, 255, size=(len(class_names), 3))
 
# Cargar el modelo DNN
model = cv2.dnn.readNet(model='input/frozen_inference_graph.pb',
                        config='input/ssd_mobilenet_v2_coco_2018_03_29.pbtxt.txt',framework='TensorFlow')
 
# Capturar el vídeo
cap = cv2.VideoCapture('input/video_1.mp4')

# Obtener la anchura y la altura de los fotogramas de vídeo para guardar correctamente los vídeos
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

# Crear el objeto `VideoWriter()`.
out = cv2.VideoWriter('video/video_result.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 30, (frame_width, frame_height))

**Recorrido en bucle por los fotogramas de vídeo y detección de objetos en cada uno de ellos**

In [8]:
# Detectar objetos en cada fotograma del vídeo
while cap.isOpened():
   ret, frame = cap.read()
   if ret:
       image = frame
       image_height, image_width, _ = image.shape
       
       # Crear una mancha a partir de una imagen
       blob = cv2.dnn.blobFromImage(image=image, size=(300, 300), mean=(104, 117, 123), swapRB=True)
      
      # hora de inicio para calcular los FPS
       start = time.time()
       model.setInput(blob)
       output = model.forward()       
       
       # Tiempo de finalización después de la detección
       end = time.time()
       
       # Calcular los FPS para la detección de fotogramas actual
       fps = 1 / (end-start)
       
       # bucle sobre cada una de las detecciones
       for detection in output[0, 0, :, :]:
           
           # extract the confidence of the detection
           confidence = detection[2]
          
           # draw bounding boxes only if the detection confidence is above...
           # ... a certain threshold, else skip
           if confidence > .4:
               
               # get the class id
               class_id = detection[1]
              
               # map the class id to the class
               class_name = class_names[int(class_id)-1]
               color = COLORS[int(class_id)]
              
               # get the bounding box coordinates
               box_x = detection[3] * image_width
               box_y = detection[4] * image_height
               
               # get the bounding box width and height
               box_width = detection[5] * image_width
               box_height = detection[6] * image_height
               
               # draw a rectangle around each detected object
               cv2.rectangle(image, (int(box_x), int(box_y)), (int(box_width), int(box_height)), color, thickness=2)
              
               # put the class name text on the detected object
               cv2.putText(image, class_name, (int(box_x), int(box_y - 5)), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
               
               # put the FPS text on top of the frame
               cv2.putText(image, f"{fps:.2f} FPS", (20, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
      
       cv2.imshow('image', image)
       out.write(image)
       if cv2.waitKey(10) & 0xFF == ord('q'):
           break
   else:
       break
 
cap.release()
cv2.destroyAllWindows()